In [1]:
'''
Trend Analysis
1. Transform description of companies into clean word tokens. (bigram 포함)
2. Run word2vec
3. Given a keyword, get related words, and plot distribution of companies.
4. Automatically extract important keywords
Approach 4-1 word network 
Approach 4-2 temporal analysis
Approach 4-3 topic model
'''

'\nTrend Analysis\n1. Transform description of companies into clean word tokens. (bigram \xed\x8f\xac\xed\x95\xa8)\n2. Run word2vec\n3. Given a keyword, get related words, and plot distribution of companies.\n4. Automatically extract important keywords\n'

In [1]:
import pandas as pd
import re
import gensim
import pickle
from datetime import datetime
from matplotlib import pyplot as plt
import networkx as nx
import operator
%matplotlib inline

In [2]:
# load data
data_folder = '../data/csv_export/'
util_folder = '../util/'
df_organizations = pd.read_csv(data_folder + 'organizations.csv'.format(data_folder), dtype={'first_funding_on': str, 'last_funding_on':str})
df_description = pd.read_csv(data_folder + 'organization_descriptions.csv')

In [4]:
# 1. Transform description of companies into clean word tokens. (bigram 포함)
# A. 문장 전처리 Helper function to clean string into list of word tokens

# input: string
# output: list of clean word tokens
def clean(string):
    # remove non-alphabet
    string = re.sub("[^a-zA-Z]", " ", string)
    # lower case
    string = string.lower()
    # remove word with length 1
    string = [word for word in string.split() if len(word) > 1]    
    return string

In [5]:
# 1. Transform description of companies into clean word tokens. (bigram 포함)
# B. create corpus, and learn bigrams

print('reading descriptions')
sentences = []
# company description is in organization_descriptions.csv file
for index, row in df_description.iterrows():
    try:
        token_list = clean(row['description'])
    except TypeError:
        print(row['description'])
    sentences.append(token_list)
    if index % 50000 == 0: print('{} / {}'.format(index, df_description.shape[0]))

# use gensim to learn bigrams from data
# they contain important information such as [deep_learning, artificial_intelligence]
print('learning bigrams')
bigram_transformer = gensim.models.Phrases(sentences)
print('getting bigram sentences')
bigram_sentences = bigram_transformer[sentences]

reading descriptions
0 / 346275
50000 / 346275
nan
nan
100000 / 346275
150000 / 346275
200000 / 346275
250000 / 346275
300000 / 346275
nan
learning bigrams
getting bigram sentences


/Users/joonheekim/Projects/futureplay/venv/lib/python2.7/site-packages/gensim/models/phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [6]:
# 1. Transform description of companies into clean word tokens. (bigram 포함)
# C. dictionary 를 만들고 저장한다

# create dictionary
print('creating dictionary of {} documents'.format(len(sentences)))
dictionary = gensim.corpora.dictionary.Dictionary()
dictionary.add_documents(bigram_sentences)

# filter extreme (words too frequent, and too infrequent)
print('before: {}'.format(dictionary))
dictionary.filter_extremes(no_above=0.1, no_below=10)
print('after: {}'.format(dictionary))

# save dictionary
dictionary.save(util_folder + 'dictionary')

creating dictionary of 346275 documents
before: Dictionary(434521 unique tokens: [u'must_pass', u'sowela', u'mdbg', u'soscharger', u'spiderg']...)
after: Dictionary(57082 unique tokens: [u'must_pass', u'cleveland_oh', u'clinical_laboratory', u'localizes', u'standardized_tests']...)


In [7]:
# 1. Transform description of companies into clean word tokens. (bigram 포함)
# D. 단어와 회사의 매핑을 만든다. 추후에 단어가 주어졌을때 관련된 회사를 찾을때 사용된다 (반대 경우도)

word2company = {}
company2word = {}
for index, row in df_description.iterrows():
    company = row[0]
    if index % 50000 == 0: print('{} / {}'.format(index, df_description.shape[0]))

    # 회사의 설명에 쓰인 문장을 갖고 온다
    try:
        sentence = clean(row['description'])
    except TypeError:
        print(row['description'])

    # bigram 도 포함하게 변환
    bigram_sentence = bigram_transformer[sentence]
    
    # keep track of company <-> word mapping
    company2word[company] = []
    for word in bigram_sentence:
        if word not in word2company: word2company[word] = []
        word2company[word].append(company)
        company2word[company].append(word)

# 겹치는 경우 뺀다
for word, _list in word2company.items():
    _list = list(set(_list))
    word2company[word] = _list
for company, _list in company2word.items():
    _list = list(set(_list))
    company2word[company] = _list
    
f = open(util_folder + 'word2company.pickle', 'w')
pickle.dump(word2company, f)
f.close()

f = open(util_folder + 'company2word.pickle', 'w')
pickle.dump(company2word, f)
f.close()

0 / 346275
50000 / 346275
nan
nan
100000 / 346275
150000 / 346275
200000 / 346275
250000 / 346275
300000 / 346275
nan


In [8]:
# 2. Run word2vec
# A. run word2vec to learn word similarity

# transform corpus to include bigrams
bigram_sentences_for_word2vec = []
count = 0
for sentence in sentences:
    if count % 50000 == 0: print count, len(sentences)
    bigram_sentence = bigram_transformer[sentence]
    bigram_sentences_for_word2vec.append(bigram_sentence)
    count += 1

# run word2vec and save result
model = gensim.models.word2vec.Word2Vec(bigram_sentences_for_word2vec, iter = 30)
model.save(util_folder + 'word2vec')

0 346275
50000 346275
100000 346275
150000 346275
200000 346275
250000 346275
300000 346275


In [10]:
# 3. Given a keyword, get related words, and plot distribution of companies.
# A.
# input:
# words = keyword (str) or keywords (list)
# condition = founded_on (str) or first_funding_on (str) (founed_on 일 경우 데이터가 정확하지 않음. 1월과 1일에 몰려 있음)
# expand (bool) = 키워드를 확장할 것인가

def plot_trend(words, condition = 'founded_on', expand = False):
    # load required dataset
    if 'word2company' not in globals():
        print('loading word2company')
        global word2company
        word2company = pickle.load(open(util_folder + 'word2company.pickle'))
    if 'model' not in globals():
        print('loading word2vec model')
        global model
        model = gensim.models.word2vec.Word2Vec.load(util_folder + 'word2vec')
        
    # expand word set if necessary
    # train 된 word2vec 을 사용하여 관련도가 높은 단어들을 포함한다
    final_words = []
    if type(words) == str:
        words = [words]
    for word in words:
        if ' ' in word:
            word = word.replace(' ', '_')
        if expand:
            _words = model.most_similar(word)
            _words = [str(_word) for _word, sim in _words]
            final_words.extend(_words)
        final_words.append(word)
    final_words = list(set(final_words))

    # choose companies that have relevant words
    companies = []
    for word in final_words:
        _companies = word2company.get(word, [])
        companies.extend(_companies)
    companies = list(set(companies))

    # 회사를 시기 별로 정리한다
    dates = df_organizations[df_organizations['uuid'].isin(companies)][condition].values
    new_dates = []
    for date in dates:
        try:
            date = datetime.strptime(date, '%Y-%m-%d')
            new_dates.append(date)
        except Exception as e:
            pass
    print('target word: {}, unique companies: {}, out of which {} meet condition,'.format(final_words, len(companies), len(new_dates)))
    df = pd.DataFrame({'dates':new_dates})
    df.dates = pd.to_datetime(df.dates)

    # 1년 단위로 plot 한다
#     df.groupby([df["dates"].dt.year]).count().plot(kind='bar', figsize = (8, 4))
    # 3개월 단위로 plot 한다
    df.groupby([df["dates"].dt.year, df["dates"].dt.quarter]).count().plot(kind='bar', figsize = (15, 4))

In [11]:
# 3. Given a keyword, get related words, and plot distribution of companies.
# B. usage examples

# plot_trend('deep learning')
# plot_trend(['deep learning'], condition = 'founded_on', expand = False)
# plot_trend(['deep learning', 'neural network', 'computer vision', 'neural networks', 'machine learning', 'artificial intelligence'], condition = 'first_funding_on', expand = False)
# plot_trend('fintech', condition = 'first_funding_on', expand = False)
# plot_trend('artificial intelligence')
plot_trend('artificial intelligence', condition = 'first_funding_on', expand = True)
# plot_trend('connected car', condition = 'first_funding_on', expand = False)

loading word2company


KeyboardInterrupt: 

In [6]:
# 4. Automatically extract important keywords
# Approach 1: build word network, run pagerank to get words with high centrality

# get data
bigram_sentences_for_wordrank = []
count = 0
df = {}
for sentence in sentences:
    if count % 50000 == 0: print count, len(sentences)
    bigram_sentence = bigram_transformer[sentence]
    word_list = list(set(bigram_sentence))
    for word in word_list:
        df[word] = df.get(word, 0) + 1
    bigram_sentences_for_wordrank.append(bigram_sentence)
    count += 1

0 346275
10000 346275
20000 346275
30000 346275
40000 346275
50000 346275
60000 346275
70000 346275
80000 346275
90000 346275
100000 346275
110000 346275
120000 346275
130000 346275
140000 346275
150000 346275
160000 346275
170000 346275
180000 346275
190000 346275
200000 346275
210000 346275
220000 346275
230000 346275
240000 346275
250000 346275
260000 346275
270000 346275
280000 346275
290000 346275
300000 346275
310000 346275
320000 346275
330000 346275
340000 346275


In [7]:
# 4. Automatically extract important keywords
# Approach 1: build word network, run pagerank to get words with high centrality

# get words to include in the network
no_below = 50
no_above = 0.1
for no_below in [50, 20]:
    print no_below, no_above
    good_words = {}
    for word, count in df.items():
        if count > no_below and count < len(sentences) * no_above:
            good_words[word] = None
    f = open(util_folder + '{}_{}.pickles'.format(no_below, no_above), 'w')
    pickle.dump(good_words, f)
    f.close()

50 0.1
20 0.1


In [ ]:
# 4. Automatically extract important keywords
# Approach 1: build word network, run pagerank to get words with high centrality

# build network
# currently it takes too long + takes too much memory 20gb+

no_below = 50
no_above = 0.1
good_words = pickle.load(open(util_folder + '{}_{}.pickles'.format(no_below, no_above)))
print(len(good_words))
g = nx.Graph()
count = 0
for sentence in bigram_sentences_for_wordrank:
    if count % 1000 == 0: print count, len(sentences)
    count += 1
    for i, word1 in enumerate(sentence):
        for j, word2 in enumerate(sentence):
            if i == j: continue
            if word1 in good_words and word2 in good_words:
                g.add_edge(word1, word2)

central = nx.degree_centrality(g)
central = sorted(central.items(), key=operator.itemgetter(1))
central.reverse()
words = [word for word, value in central]
print words[:100]
f = open(util_folder + 'keywords_network.pickle')
pickle.dump(central, f)
f.close()

19013
0 346275
1000 346275
2000 346275
3000 346275
4000 346275
5000 346275
6000 346275
7000 346275
8000 346275
9000 346275
10000 346275
11000 346275
12000 346275
13000 346275
14000 346275
15000 346275
16000 346275
17000 346275
18000 346275
19000 346275
20000 346275
21000 346275
22000 346275
23000 346275
24000 346275
25000 346275
26000 346275
27000 346275
28000 346275
29000 346275
30000 346275
31000 346275
32000 346275
33000 346275
34000 346275
35000 346275
36000 346275
37000 346275
38000 346275
39000 346275
40000 346275
41000 346275
42000 346275
43000 346275
44000 346275
45000 346275
46000 346275
47000 346275
48000 346275
49000 346275
50000 346275
51000 346275
52000 346275
53000 346275
54000 346275
55000 346275
56000 346275
57000 346275
58000 346275
59000 346275
60000 346275
61000 346275
62000 346275
63000 346275
64000 346275
65000 346275
66000 346275
67000 346275
68000 346275
69000 346275
70000 346275
71000 346275
72000 346275
73000 346275
74000 346275
75000 346275
76000 346275
77000 

In [12]:
# 4. Automatically extract important keywords
# Approach 2: 기간에 따라 급격히 달라지는 단어의 흐름으로 키워드 찾아내기
# 2-1. keyword1: 특정 기간 전후로 사용 비율이 급격히 높은 단어들 보기
# 2-2. keyword2: 특저 기간 내에 사용 비율이 다른 때 보다 급격히 높은 단어들 보기

WORD_MIN = 10 # minimum number of occurrences to be considered as a keyword
def keyword1(before_after):
    # load required dataset
    if 'word2company' not in globals():
        print('loading word2company')
        global word2company
        word2company = pickle.load(open(util_folder + 'word2company.pickle'))
    if 'company2word' not in globals():
        print('loading company2word')
        global company2word
        company2word = pickle.load(open(util_folder + 'company2word.pickle'))
    if 'model' not in globals():
        print('loading word2vec model')
        global model
        model = gensim.models.word2vec.Word2Vec.load(util_folder + 'word2vec')
        
    # 기간 전의 회사들, 기간 후의 회사들
    companies_before = df_organizations[df_organizations['founded_on'] < before_after]
    companies_after = df_organizations[df_organizations['founded_on'] >= before_after]
    print('Extracting keywords from startups after {}'.format(before_after))
    print('before: {} after: {}'.format(companies_before.shape[0], companies_after.shape[0]))
    
    # 기간 전의 회사들, 기간 후의 회사들로부터 각 단어의 document frequency (DF) 를 측정한다.
    df_before = {}
    df_after = {}
    for company in companies_before['uuid'].values:
        words = company2word.get(company, [])
        words = list(set(words))
        for word in words:
            df_before[word] = df_before.get(word, 0) + 1
    for company in companies_after['uuid'].values:
        words = company2word.get(company, [])
        words = list(set(words))
        for word in words:
            df_after[word] = df_after.get(word, 0) + 1
            
    # 각 단어의 사용 비율을 측정한다 (전체 문서에서 각 단어가 사용될 확률)
    ratio_dict = {}
    for word, count in df_after.items():
        if count >= WORD_MIN:
            # smoothing factor 포함
            smooth = 0.00001
            ratio_before = df_before.get(word, 0) / float(len(companies_before['uuid'].values)) + smooth
            ratio_after = df_after.get(word, 0) / float(len(companies_after['uuid'].values)) + smooth
            ratio_dict[word] = ratio_after / ratio_before

    # 결과를 sorting 해서 리스트의 형태로
    # [(keyword1, score1), (keyword2, score2), ...]
    ratio_dict = sorted(ratio_dict.items(), key=operator.itemgetter(1))
    ratio_dict.reverse()
    print ratio_dict[:50]
    f = open(util_folder + 'keywords1_' + before_after + '_ratio.pickle', 'w')
    pickle.dump(ratio_dict, f)
    f.close()

In [13]:
# 4. Automatically extract important keywords
# Approach 2-1: 사용 예제

for year in range(2000, 2017):
    keyword1('{}-01-01'.format(year))

Extracting keywords from startups after 2000-01-01
before: 92190 after: 286885
[(u'blockchain', 90.23436220088189), (u'tech_startup', 72.10863237882775), (u'kickstarter', 50.148613555954476), (u'gamified', 44.92003764574655), (u'bitcoin', 44.19722163698627), (u'public_beta', 43.52575073635777), (u'bots', 40.737176917580214), (u'hashtags', 40.04003346288582), (u'bitcoins', 38.29717482614985), (u'crowd_sourcing', 37.60003137145546), (u'meetups', 37.251459644108266), (u'coworking_space', 36.554316189413875), (u'digital_currency', 35.85717273471949), (u'foodies', 34.11431409798351), (u'super_easy', 33.76574237063632), (u'cashback', 32.72002718859473), (u'private_beta', 31.579519866848685), (u'hashtag', 30.97716855185876), (u'earn_rewards', 29.234309915122786), (u'blogging_platform', 28.88573818777559), (u'korean_startup', 28.88573818777559), (u'anonymity', 28.88573818777559), (u'dating_app', 27.840023005734007), (u'be_notified', 27.840023005734007), (u'recommendation_engine', 27.0650272101

Extracting keywords from startups after 2005-01-01
before: 130371 after: 248704
[(u'blockchain', 58.59030971541255), (u'tech_startup', 46.757884653547485), (u'hashtag', 35.57925887802367), (u'korean_startup', 33.166752444673186), (u'crowdfunding_platform', 28.68284579266278), (u'get_inspired', 28.341739577972206), (u'foodie', 27.537570766855378), (u'take_picture', 25.92923314462172), (u'hashtags', 25.823594159478525), (u'demo_day', 25.527148739063303), (u'gifs', 25.527148739063303), (u'bitcoin', 25.02966570195744), (u'bitcoins', 24.68586098045304), (u'crowdfund', 24.320895522388057), (u'meetups', 24.003221073037746), (u'combinator', 23.243178212785256), (u'push_notifications', 23.079943371949994), (u'incentivized', 22.310473494595985), (u'leaderboards', 21.90838908903757), (u'indiegogo', 21.506304683479154), (u'techcrunch_disrupt', 21.506304683479154), (u'get_rewarded', 21.506304683479154), (u'sharing_economy', 21.339131694039143), (u'curated_marketplace', 21.104220277920742), (u'selfi

Extracting keywords from startups after 2010-01-01
before: 197307 after: 181768
[(u'take_picture', 35.10941419831874), (u'blockchain', 34.283543688759195), (u'bitcoin', 24.802317704437588), (u'smart_contracts', 24.1063773601514), (u'blockchain_technology', 22.6267359175261), (u'gifs', 22.570055761501905), (u'facebook_messenger', 22.570055761501905), (u'curated_list', 20.805466308701202), (u'bitcoins', 19.048553550445), (u'curated_marketplace', 18.553880884403295), (u'sharing_economy', 17.67393815975654), (u'cryptocurrencies', 17.09345365636744), (u'fashion_bloggers', 16.404251573434266), (u'wearable_tech', 15.8540997315259), (u'cryptocurrency', 15.777407271420222), (u'emojis', 15.303947889617532), (u'jabong', 15.303947889617532), (u'swipe_right', 15.303947889617532), (u'angelpad_org', 15.26791962132262), (u'angelpad_http', 15.26791962132262), (u'fintech_startup', 14.902812814313654), (u'equity_crowdfunding', 14.147964932524756), (u'facebook_instagram', 14.141834131907833), (u'korean_st

Extracting keywords from startups after 2015-01-01
before: 333220 after: 45855
[(u'gig_economy', 16.592601387330195), (u'robo_advisor', 13.338541117683244), (u'facebook_messenger', 12.912275404496002), (u'chat_bot', 11.737698468961028), (u'ai_powered', 11.654604978191765), (u'chatbot', 11.261994309321572), (u'conversational_commerce', 11.1953661663382), (u'vr_experiences', 10.60603404642355), (u'smart_contracts', 9.945557275744829), (u'htc_vive', 9.866019669954932), (u'ethereum', 9.866019669954932), (u'slack', 8.968292716184092), (u'mixed_reality', 8.863422266640288), (u'sales_prospecting', 8.143888174037377), (u'immuno_oncology', 7.880745151201772), (u'fully_autonomous', 7.80066194990951), (u'fintech_startup', 7.650510733963377), (u'chatbots', 7.555712593144184), (u'blockchain', 7.330105819206438), (u'vr_ar', 7.292805889871863), (u'ai', 6.580796563673403), (u'paytm', 6.316827274340433), (u'streamers', 6.316827274340433), (u'vr', 6.18041588941974), (u'amazon_alexa', 6.16276035239062), 

In [14]:
# 4. Automatically extract important keywords
# Approach 2: 기간에 따라 급격히 달라지는 단어의 흐름으로 키워드 찾아내기
# 2-1. keyword1: 특정 기간 전후로 사용 비율이 급격히 높은 단어들 보기
# 2-2. keyword2: 특저 기간 내에 사용 비율이 다른 때 보다 급격히 높은 단어들 보기

WORD_MIN = 10 # minimum number of occurrences to be considered as a keyword
# word2company = pickle.load(open(util_folder + 'word2company.pickle'))
# company2word = pickle.load(open(util_folder + 'company2word.pickle'))
def keyword2(after, before):
    # load required dataset
    if 'word2company' not in globals():
        print('loading word2company')
        global word2company
        word2company = pickle.load(open(util_folder + 'word2company.pickle'))
    if 'company2word' not in globals():
        print('loading company2word')
        global company2word
        company2word = pickle.load(open(util_folder + 'company2word.pickle'))
    if 'model' not in globals():
        print('loading word2vec model')
        global model
        model = gensim.models.word2vec.Word2Vec.load(util_folder + 'word2vec')
        
    # 기간 사이에 속하는 회사들과 그렇지 않은 회사들 분류
    companies_yes = df_organizations[df_organizations['founded_on'] < before]
    companies_yes = companies_yes[companies_yes['founded_on'] >= after]
    companies_no = df_organizations[~df_organizations['uuid'].isin(companies_yes['uuid'].values)]
    print('Extracting keywords from startups after {} and no {}'.format(after, before))
    print('no: {} yes: {}'.format(companies_no.shape[0], companies_yes.shape[0]))

    # 각 단어의 document frequency (DF) 측정
    df_no = {}
    df_yes = {}
    for company in companies_no['uuid'].values:
        words = company2word.get(company, [])
        words = list(set(words))
        for word in words:
            df_no[word] = df_no.get(word, 0) + 1
    for company in companies_yes['uuid'].values:
        words = company2word.get(company, [])
        words = list(set(words))
        for word in words:
            df_yes[word] = df_yes.get(word, 0) + 1
            
    # 각 단어의 사용 비율 측정
    ratio_dict = {}
    for word, count in df_yes.items():
        if count >= WORD_MIN:
            smoothing = 0.00000001
            ratio_no = df_no.get(word, 0) / float(len(companies_no['uuid'].values)) + smoothing
            ratio_yes = df_yes.get(word, 0) / float(len(companies_yes['uuid'].values)) + smoothing
#             print word, ratio_no, ratio_yes
            ratio_dict[word] = ratio_yes / ratio_no

    # 저장
    ratio_dict = sorted(ratio_dict.items(), key=operator.itemgetter(1))
    ratio_dict.reverse()
    print ratio_dict[:50]
    f = open(util_folder + 'keywords2_' + after + '_' + before + '_ratio.pickle', 'w')
    pickle.dump(ratio_dict, f)
    f.close()

In [15]:
# 4. Automatically extract important keywords
# Approach 2-2: 사용 예제

for year in range(2010, 2017):
    keyword2('{}-01-01'.format(year), '{}-01-01'.format(year + 1))

Extracting keywords from startups after 2010-01-01 and no 2011-01-01
no: 498359 yes: 22049
[(u'timing_mon', 20.98433044955354), (u'collective_buying', 11.927500178359873), (u'group_buying', 10.833774599554868), (u'daily_deal', 10.739774873630665), (u'fri_am', 9.970305861947862), (u'daily_deals', 9.062069540268055), (u'puma', 7.136776308663427), (u'enticing', 7.102723691180179), (u'groupon', 6.854568952374305), (u'clipping', 6.45703447998327), (u'android_blackberry', 6.344049321478029), (u'flash_sales', 5.739901938842894), (u'kindle', 5.563303111434655), (u'loyalty_rewards', 5.534805781500197), (u'foursquare', 5.460818911585524), (u'core_strength', 5.2558649519968), (u'seo_friendly', 5.215601458516081), (u'ipad_iphone', 5.183831702319844), (u'techcrunch_disrupt', 5.136427003764247), (u'exclusive_license', 5.073580441006458), (u'display_ads', 5.022371056631115), (u'microsoft_azure', 5.022296820970324), (u'news_aggregator', 4.972118945036263), (u'twitter_account', 4.8430850181821725), (u'

Extracting keywords from startups after 2015-01-01 and no 2016-01-01
no: 493308 yes: 27100
[(u'fully_autonomous', 12.132447196502877), (u'immuno_oncology', 10.704976288252658), (u'grocery_delivery', 9.099958710223069), (u'snapdeal', 9.099625811131343), (u'subscription_box', 8.704248429710955), (u'fintech_startup', 8.158858579312406), (u'sitters', 7.912972612077828), (u'uses_machine', 7.644708260089904), (u'vr_experiences', 7.58333317564588), (u'ethereum', 7.414965704376986), (u'facebook_messenger', 7.358092179566065), (u'curated_marketplace', 7.078207693605651), (u'ai_powered', 7.0459372336659225), (u'mins', 7.000110511521579), (u'sharing_economy', 6.93429157108409), (u'group_chats', 6.673591042623167), (u'slack', 6.525435258677193), (u'ends_up', 5.871240067992536), (u'gifs', 5.861656177595214), (u'personal_assistant', 5.532140262707401), (u'whatsapp', 5.381641660132333), (u'neighbours', 5.353264627119309), (u'home_cooked', 5.200335763782104), (u'gupta', 5.1618084609386825), (u'deep_le

In [10]:
# 4. Automatically extract important keywords
# Approach 3. LDA (topic model) 을 이용하여 중요한 토픽들을 찾아내고 각 단어의 상위 단어들을 키워드로 사용한다

count = 0
bigram_sentences_for_lda = []
for sentence in sentences:
    if count % 50000 == 0: print count, len(sentences)
    bigram_sentence = bigram_transformer[sentence]
    bigram_sentence = dictionary.doc2bow(bigram_sentence)
    bigram_sentences_for_lda.append(bigram_sentence)
    count += 1

0 346275
50000 346275
100000 346275
150000 346275
200000 346275
250000 346275
300000 346275


In [8]:
# %time model = gensim.models.ldamodel.LdaModel(corpus = bigram_sentences_for_lda, id2word = dictionary)
# model = gensim.models.ldamodel.LdaModel.load(util_folder + 'topic_500')
for i in range(100):
    print model.print_topic(i)

0.041*"projects" + 0.038*"project" + 0.026*"over" + 0.023*"has_been" + 0.022*"since" + 0.020*"first" + 0.019*"now" + 0.018*"one" + 0.018*"currently" + 0.016*"year"
0.052*"commercial" + 0.045*"house" + 0.042*"agents" + 0.035*"booking" + 0.018*"guides" + 0.018*"residential" + 0.018*"guests" + 0.018*"bar" + 0.018*"all_types" + 0.017*"quick"
0.201*"app" + 0.095*"llc" + 0.048*"mobile_app" + 0.046*"developer" + 0.042*"apps" + 0.041*"has_developed" + 0.034*"over_million" + 0.034*"ios" + 0.028*"original" + 0.024*"apple"
0.097*"tech" + 0.048*"natural" + 0.026*"player" + 0.026*"photography" + 0.025*"senior" + 0.025*"near" + 0.023*"organic" + 0.021*"certain" + 0.021*"found" + 0.021*"lets"
0.053*"face" + 0.045*"cash" + 0.043*"machine" + 0.042*"contacts" + 0.040*"starting" + 0.036*"changes" + 0.035*"hub" + 0.030*"rapid" + 0.025*"machines" + 0.024*"manner"
0.105*"marketing" + 0.039*"social_media" + 0.033*"web" + 0.029*"advertising" + 0.022*"sales" + 0.020*"search_engine" + 0.020*"clients" + 0.019*"a

0.402*"com" + 0.063*"www" + 0.062*"http_www" + 0.026*"website" + 0.015*"gifts" + 0.014*"child" + 0.013*"gift" + 0.012*"info" + 0.011*"south" + 0.011*"florida"
0.070*"payment" + 0.040*"automation" + 0.038*"electronic" + 0.038*"labs" + 0.037*"payments" + 0.035*"models" + 0.033*"automotive" + 0.032*"merchants" + 0.028*"box" + 0.028*"cards"
0.060*"firm" + 0.060*"new_york" + 0.054*"million" + 0.048*"private" + 0.029*"venture_capital" + 0.029*"invest" + 0.029*"private_equity" + 0.027*"capital" + 0.018*"credit" + 0.018*"fund"
0.066*"total" + 0.056*"class" + 0.044*"plus" + 0.043*"owner" + 0.030*"cases" + 0.030*"sourcing" + 0.026*"included" + 0.024*"july" + 0.022*"defense" + 0.022*"semiconductor"
0.060*"training" + 0.048*"program" + 0.038*"professional" + 0.035*"programs" + 0.020*"career" + 0.017*"skills" + 0.014*"assistance" + 0.012*"professionals" + 0.009*"coaching" + 0.009*"support"
0.031*"performance" + 0.023*"increase" + 0.023*"relationships" + 0.021*"value" + 0.021*"sustainable" + 0.019*"